In [1]:
import numpy as np
import matplotlib.pyplot as plt

from neurnet import NeuralNetwork

In [3]:
"""
    EXAMPLE 1:  one-to-one fit numerical data
""" 

def scalar_function(X):
    return np.sin(X)

n_inputs = 100
noise = 0.2
X_test = np.linspace(-10,10, n_inputs).reshape(n_inputs, 1)
Y_test = scalar_function(X_test) + noise*np.random.randn(n_inputs,1)



# Create the network
topology = [2,3,4]
activation_funcs = ['relu',
                    'relu',
                    'linear']
cost_funct = 'mse'

ryc = NeuralNetwork(n_inputs, topology, activation_funcs, cost_funct)
cost = ryc.train(X_test, Y_test)


# Plot
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 16}

plt.rc('text', usetex=True)
plt.rc('font', **font)
fig, ax = plt.subplots()
        
plt.plot(X_test, Y_test, 'o')
plt.plot(X_test, scalar_function(X_test),
          color="#aaaaaa",
          label="ground truth")

ax.set_xlabel(r"$x$")
ax.set_ylabel(r"$y$")

(100, 1)
(100, 1)


  0%|          | 0/4 [00:00<?, ?it/s]

AttributeError: 'NeuralNetwork' object has no attribute 'n_neurons'